## Préparation des données et des modèles

In [8]:
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'


In [9]:
from sklearn import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import seaborn as sns
import regex as re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split


### Chargement et préparation des données

In [49]:

# Charger les données par chunks
print("Début du chargement des données...")
df = pd.read_csv('EFREI - LIPSTIP - 50k elements EPO.csv', chunksize=10000)
print("Chargement des données terminé.")

df_list = []
for chunk in tqdm(df, desc="Chargement des chunks"):
    df_list.append(chunk)

df = pd.concat(df_list)
print("Concaténation des chunks terminée.")

# Sélectionner les colonnes nécessaires
df_2 = df.drop(columns=['Date d\'application', 'Numero de publication', 'date de publication', 'IPC'])
print("Colonnes inutiles supprimées.")

# Définir la fonction d'extraction d'information essentielle
def clean_xml(xml_text):
    if not xml_text:
        return []
    xml_text = re.sub(r'<!--.*?-->', '', xml_text, flags=re.DOTALL)
    xml_text = re.sub(r'<[^>]+>', '', xml_text, flags=re.DOTALL)
    xml_text = re.sub(r'</?p[^>]*>', '', xml_text, flags=re.DOTALL)
    paragraphs = xml_text.split('\n')
    essential_info = [paragraph.strip() for paragraph in paragraphs if paragraph.strip()]
    return essential_info

# Assurer qu'il n'y a pas de valeurs nulles dans 'description'
if df_2['description'].isnull().values.any():
    print("Il y a des valeurs nulles dans la colonne 'description'.")
    df_2['description'] = df_2['description'].fillna('')

# Créer la nouvelle colonne pour stocker les informations essentielles extraites
print("Début de l'extraction des informations essentielles...")
df_2['infos_essentielles'] = df_2['description'].apply(clean_xml)
print("Extraction des informations essentielles terminée.")

# Afficher les premières lignes pour vérifier les résultats
df_2.head(50)


Début du chargement des données...
Chargement des données terminé.


Chargement des chunks: 5it [00:54, 10.99s/it]


Concaténation des chunks terminée.
Colonnes inutiles supprimées.
Début de l'extraction des informations essentielles...
Extraction des informations essentielles terminée.


,Numéro d'application,CPC,claim,description,infos_essentielles
0,17167919,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...",[The present invention relates to a cover wind...
1,17166775,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE...",[FIELD OF THE INVENTIONThe present disclosure ...
2,17166069,"['D06F58-206', 'D06F58-20']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>...",[FIELDThe present invention relates to a techn...
3,18771363,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec...",[Technical FieldThe present invention relates ...
4,19881950,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...",[The application claims the priority to Chines...
5,19171424,"['F16D41-22', 'F16D43-22', 'F16D43-06', 'F16D1...","<!-- EPO <DP n=""7""> --><claim id=""c-en-0001"" n...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>...",[BACKGROUNDThis invention relates to a synchro...
6,18159458,"['C25B15-00', 'C25B1-04', 'Y02E10-56', 'Y02E60...","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE...",[FIELD OF THE INVENTIONThe present invention i...
7,18174621,"['H01J37-28', 'H01J37-147', 'H01J37-3177', 'H0...","<!-- EPO <DP n=""25""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">BAC...",[BACKGROUNDField of the InventionThe present i...
8,19888805,"['F24F11-79', 'F24F11-65', 'F24F11-64', 'F24F1...","<!-- EPO <DP n=""43""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>...",[PRIORITY INFORMATIONThis application claims p...
9,17305326,"['H04L5-0094', 'H04L5-0007', 'H04L5-0039', 'H0...","<!-- EPO <DP n=""23""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...",[The present invention generally relates to re...


In [34]:
# La colonne CPC contient des listes de codes CPC, par exemple : ['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G06F1-1626', 'G06F1-1656', 'G06F1-1637', 'G02F1-133308', 'B23K2101-34', 'B23K2101-40', 'B23K2103-42', 'B23K2103-172', 'G02F1-133331']
# j'aimerais vérifier la premiere lettre de chaque code CPC de chaque brevet pour voir toutes les lettres possibles:
# je veux une liste finale de toutes les lettres possibles :

# Créer un ensemble pour stocker toutes les lettres possibles
all_first_letters = set()

# Itérer à travers chaque ligne de la colonne 'CPC'
for cpc_list in df_2['CPC']:
    # Évaluer la chaîne de caractères en tant que liste
    cpc_list = eval(cpc_list)
    if isinstance(cpc_list, list):
        for code in cpc_list:
            if len(code) > 0:
                all_first_letters.add(code[0])

# Convertir l'ensemble en liste pour obtenir une liste finale de toutes les lettres possibles
all_first_letters = sorted(list(all_first_letters))

print("Toutes les lettres possibles en tant que premières lettres des codes CPC :", all_first_letters)


Toutes les lettres possibles en tant que premières lettres des codes CPC : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']


In [50]:
# afficher nombre de lignes
df_2.shape

(50000, 5)

In [51]:
# Fonction pour compter le nombre de mots
def count_words(text):
    if isinstance(text, list):
        text = ' '.join(text)
    return len(text.split())

# Afficher le nombre de mots de la première ligne dans la colonne 'infos_essentielles'
first_row_words_count = count_words(df_2['infos_essentielles'].iloc[0])
print("Nombre de mots dans la première ligne de la colonne 'infos_essentielles':", first_row_words_count)

Nombre de mots dans la première ligne de la colonne 'infos_essentielles': 8581


In [40]:
# Charger le tokenizer et le modèle DistilBERT
from transformers import DistilBertTokenizer, DistilBertModel
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', force_download=True)
model = DistilBertModel.from_pretrained('distilbert-base-uncased', force_download=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [48]:
sample_df = df_2.head(10000).copy()

# Fonction pour obtenir les embeddings de BERT
def get_bert_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Appliquer la fonction aux descriptions
sample_texts = sample_df['description'].tolist()
embeddings = get_bert_embeddings(sample_texts)

# Ajouter les embeddings au dataframe en utilisant .loc pour éviter les avertissements
sample_df.loc[:, 'embeddings'] = list(embeddings)

# Afficher les premières lignes pour vérifier les résultats
print(sample_df.head())



KeyboardInterrupt



In [ ]:

# Vérifier quelques exemples de la colonne 'CPC'
print(sample_df['CPC'].head())

# Fonction pour extraire la première lettre du premier code CPC pour chaque brevet
def extract_first_letter(cpc_list):
    # Convertir les chaînes de caractères en listes de chaînes
    cpc_list = eval(cpc_list)
    if isinstance(cpc_list, list) and len(cpc_list) > 0:
        return cpc_list[0][0]  # Extraire la première lettre du premier code CPC
    return ''

# Appliquer la fonction pour extraire les labels
sample_df['label'] = sample_df['CPC'].apply(extract_first_letter)

# Afficher les étiquettes pour vérifier
print("Étiquettes extraites :", sample_df['label'].unique())


In [ ]:

# Évaluation de la cohérence des embeddings
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Réduire les dimensions pour visualisation
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(list(sample_df['embeddings']))

# Créer un DataFrame pour les embeddings réduits
embedding_df = pd.DataFrame(reduced_embeddings, columns=['PC1', 'PC2'])
embedding_df['label'] = sample_df['label']

# Visualiser les embeddings
plt.figure(figsize=(10, 7))
sns.scatterplot(data=embedding_df, x='PC1', y='PC2', hue='label', palette='Set1')
plt.title('Visualisation des Embeddings BERT Réduits')
plt.show()


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Préparer les données
X = np.array(sample_df['embeddings'].tolist())
y = sample_df['label']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un modèle de régression logistique
log_reg_model = LogisticRegression(max_iter=1000)
log_reg_model.fit(X_train, y_train)

# Faire des prédictions
y_pred = log_reg_model.predict(X_test)

# Évaluer le modèle
print(classification_report(y_test, y_pred, zero_division=1))